In [221]:
import pandas as pd
import re
from unicodedata import normalize


# Quitar tildes excepto ñ
def remove_tildes(string: str) -> str:
    string = re.sub(
        r"([^n\u0300-\u036f]|n(?!\u0303(?![\u0300-\u036f])))[\u0300-\u036f]+", r"\1", 
        normalize("NFD", string), 0, re.I
    )
    string = normalize("NFC", string)
    return string


# Quitar parentesis
def remove_parentesis(string: str) -> str:
    string = re.sub(r"\([^)]*\)", "", string)
    return string.strip()


def df_conditions(df: pd.DataFrame, col: str, values: list[str]):
    condition = (df[col] == values[0])
    if len(values) == 1:
        return condition
    
    for value in values[1:]:
        condition |= (df[col] == value)

    return condition

In [222]:
df = pd.read_csv("./raw/Directorio_de_Casas_de_Justicia_y_Centros_de_Convivencia_Ciudadana._20241103.csv", engine="python")
df

,DEPARTAMENTO,MUNICIPIO,CASA DE JUSTICIA,DIRECCIÓN,NUMERO FIJO,CORREOS INSTITUCIONALES,AÑO
0,AMAZONAS,Leticia,Leticia,Calle 9 No 9 - 103,No registra,centrodeconvivencialeticia@centrosdeconvivenci...,2020
1,ANTIOQUIA,Amalfi,Amalfi,Calle 19 Santander Nro.21-39,(4) 830 00 90,casadejusticiaamalfi@casasdejusticia.gov.co,2020
2,ANTIOQUIA,Anori,Anori,"Cra. 30 Nro. 30-42, parque principal.",(4) 8350473 - (4) 8350842 Ext. 116,casadejusticiaanori@casasdejusticia.gov.co,2020
3,ANTIOQUIA,Apartado,Apartado,Calle 103 B No. 98 - 82 Barrio Chinita,No Registra,casadejusticiaapartado@casasdejusticia.gov.co,2020
4,ANTIOQUIA,Bello,Bello,Diagonal 65 No. 42 A – 11 Barrio Niquía,6047944 ext. 1401,casadejusticiabello@casasdejusticia.gov.co,2020
...,...,...,...,...,...,...,...
1232,Valle del Cauca,Buga,Buga,Calle 30 No. 13 A - 00,(2) 2386780 -2386781,casadejusticiabuga@casasdejusticia.gov.co,2024
1233,Valle del Cauca,Florida,Florida,Calle 7a #19 - 30,No registra,casadejusticiaflorida@casasdejusticia.gov.co,2024
1234,Valle del Cauca,Palmira,Palmira,Calle 57 # 44-22 barrio Caimitos,3188276379,casadejusticiapalmira@casasdejusticia.gov.co,2024
1235,Valle del Cauca,Pradera,Pradera,Carrera 8 No.7-57,(2) 620 00 00,casadejusticiapradera@casasdejusticia.gov.co,2024


In [223]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1237 entries, 0 to 1236
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   DEPARTAMENTO             1237 non-null   object
 1   MUNICIPIO                1237 non-null   object
 2   CASA DE JUSTICIA         1237 non-null   object
 3   DIRECCIÓN                1237 non-null   object
 4   NUMERO FIJO              1232 non-null   object
 5   CORREOS INSTITUCIONALES  1233 non-null   object
 6   AÑO                      1237 non-null   int64 
dtypes: int64(1), object(6)
memory usage: 67.8+ KB


## Fix department names

In [224]:
print(f'ANTES: {df["DEPARTAMENTO"].unique()}')

# Quitar tildes
df["DEPARTAMENTO"] = df["DEPARTAMENTO"].apply(remove_tildes)

# Colocar todo en mayúsculas
df["DEPARTAMENTO"] = df["DEPARTAMENTO"].apply(str.upper)

# Acomodar san andres manualmente (Quitar "ISLA" y "_")
condition = df_conditions(df, "DEPARTAMENTO", ["SAN_ANDRES", "SAN ANDRES  ISLA"])
df.loc[condition, "DEPARTAMENTO"] = "SAN ANDRES"

print(f'\nDespués de acomodar: {df["DEPARTAMENTO"].unique()}')

ANTES: ['AMAZONAS' 'ANTIOQUIA' 'ARAUCA' 'ATLANTICO' 'CUNDINAMARCA' 'BOGOTA'
 'BOLIVAR' 'BOYACA' 'CALDAS' 'CAQUETA' 'CASANARE' 'CAUCA' 'CESAR' 'CHOCO'
 'CORDOBA' 'GUAVIARE' 'HUILA' 'LA GUAJIRA' 'Amazonas' 'MAGDALENA' 'META'
 'NARIÑO' 'NORTE DE SANTANDER' 'PUTUMAYO' 'QUINDIO' 'RISARALDA'
 'SAN_ANDRES' 'SANTANDER' 'SUCRE' 'TOLIMA' 'VALLE DEL CAUCA' 'Antioquia'
 'SAN ANDRÉS' 'Arauca' 'Atlantico' 'Bogota' 'Bolivar' 'Boyaca' 'Caldas'
 'Caqueta' 'Casanare' 'Cauca' 'Cesar' 'Choco' 'Cordoba' 'Cundinamarca'
 'Guaviare' 'Huila' 'La Guajira' 'Magdalena' 'Meta' 'Nariño'
 'Norte de Santander' 'Putumayo' 'Quindio' 'Risaralda' 'San Andres'
 'Santander' 'Sucre' 'Tolima' 'Valle del Cauca' 'QUINDÍO' 'Bogotá'
 'NORTE SANTANDER' 'SAN ANDRÉS  ISLA']

Después de acomodar: ['AMAZONAS' 'ANTIOQUIA' 'ARAUCA' 'ATLANTICO' 'CUNDINAMARCA' 'BOGOTA'
 'BOLIVAR' 'BOYACA' 'CALDAS' 'CAQUETA' 'CASANARE' 'CAUCA' 'CESAR' 'CHOCO'
 'CORDOBA' 'GUAVIARE' 'HUILA' 'LA GUAJIRA' 'MAGDALENA' 'META' 'NARIÑO'
 'NORTE DE SANTANDER' 'PUT

## Fix municipalities name

In [225]:
print(f'ANTES: {df["MUNICIPIO"].unique()}')

# Quitar tildes
df["MUNICIPIO"] = df["MUNICIPIO"].apply(remove_tildes)

# Colocar todo en mayúsculas
df["MUNICIPIO"] = df["MUNICIPIO"].apply(str.upper)

# Quitar parentesis
df["MUNICIPIO"] = df["MUNICIPIO"].apply(remove_parentesis)

print(f'\nDespués de acomodar: {df["MUNICIPIO"].unique()}')

ANTES: ['Leticia' 'Amalfi' 'Anori' 'Apartado' 'Bello' 'Caceres' 'Caucasia'
 'Chigorodo' 'Ebejico' 'El Bagre' 'El Carmen de Viboral (Antioquia)'
 'Envigado' 'Frontino' 'Itagui' 'La Ceja (Antioquia)' 'Medellin' 'Nechi'
 'Remedios' 'Rionegro (Antioquia)' 'Santafe de Antioquia' 'Segovia'
 'Sonson' 'Taraza' 'Turbo' 'Vegachi' 'Yali' 'Yarumal' 'Yondo' 'Zaragoza'
 'Arauquita' 'Fortul' 'Saravena' 'Baranoa' 'Barranquilla' 'GIRARDOT'
 'Usiacuri' 'BOGOTA' 'Cantagallo' 'Cartagena (Bolivar)' 'Magangue'
 'San Jacinto (Bolivar)' 'Chiquinquira' 'Paipa' 'Puerto Boyaca' 'Sogamoso'
 'Tunja' 'Villa de Leyva' 'La Dorada' 'Manizales' 'Marquetalia'
 'Riosucio (Caldas)' 'Supia' 'Doncello o el Doncello'
 'San Vicente del Caguan (Caqueta)' 'Florencia (Caqueta)' 'La Salina'
 'Sacama' 'Nunchia' 'Paz de Ariporo' 'Yopal' 'Buenos Aires' 'Caloto'
 'Corinto' 'Guachene' 'Piendamo' 'Popayan' 'Puerto Tejada'
 'Santander de Quilichao' 'Toribio' 'Aguachica' 'Valledupar' 'Condoto'
 'Istmina' 'Novita' 'Quibdo' 'Tado' 'Carmen 

In [226]:
df

,DEPARTAMENTO,MUNICIPIO,CASA DE JUSTICIA,DIRECCIÓN,NUMERO FIJO,CORREOS INSTITUCIONALES,AÑO
0,AMAZONAS,LETICIA,Leticia,Calle 9 No 9 - 103,No registra,centrodeconvivencialeticia@centrosdeconvivenci...,2020
1,ANTIOQUIA,AMALFI,Amalfi,Calle 19 Santander Nro.21-39,(4) 830 00 90,casadejusticiaamalfi@casasdejusticia.gov.co,2020
2,ANTIOQUIA,ANORI,Anori,"Cra. 30 Nro. 30-42, parque principal.",(4) 8350473 - (4) 8350842 Ext. 116,casadejusticiaanori@casasdejusticia.gov.co,2020
3,ANTIOQUIA,APARTADO,Apartado,Calle 103 B No. 98 - 82 Barrio Chinita,No Registra,casadejusticiaapartado@casasdejusticia.gov.co,2020
4,ANTIOQUIA,BELLO,Bello,Diagonal 65 No. 42 A – 11 Barrio Niquía,6047944 ext. 1401,casadejusticiabello@casasdejusticia.gov.co,2020
...,...,...,...,...,...,...,...
1232,VALLE DEL CAUCA,BUGA,Buga,Calle 30 No. 13 A - 00,(2) 2386780 -2386781,casadejusticiabuga@casasdejusticia.gov.co,2024
1233,VALLE DEL CAUCA,FLORIDA,Florida,Calle 7a #19 - 30,No registra,casadejusticiaflorida@casasdejusticia.gov.co,2024
1234,VALLE DEL CAUCA,PALMIRA,Palmira,Calle 57 # 44-22 barrio Caimitos,3188276379,casadejusticiapalmira@casasdejusticia.gov.co,2024
1235,VALLE DEL CAUCA,PRADERA,Pradera,Carrera 8 No.7-57,(2) 620 00 00,casadejusticiapradera@casasdejusticia.gov.co,2024


## Remove reapeted directions and emails


In [250]:
from typing import Any

def fix_emails(email: Any) -> str | None:
    try:
        new_email = str.lower(email)
    except TypeError:
        return None
    
    new_email = new_email.replace(" ", "")
    return new_email


df["CORREOS INSTITUCIONALES"] = df["CORREOS INSTITUCIONALES"].apply(fix_emails)

new_df = df.drop_duplicates(subset=["DEPARTAMENTO", "MUNICIPIO", "DIRECCIÓN"], ignore_index=True)
new_df = df.drop_duplicates(subset="CORREOS INSTITUCIONALES", ignore_index=True)
new_df

,DEPARTAMENTO,MUNICIPIO,CASA DE JUSTICIA,DIRECCIÓN,NUMERO FIJO,CORREOS INSTITUCIONALES,AÑO
0,AMAZONAS,LETICIA,Leticia,Calle 9 No 9 - 103,No registra,centrodeconvivencialeticia@centrosdeconvivenci...,2020
1,ANTIOQUIA,AMALFI,Amalfi,Calle 19 Santander Nro.21-39,(4) 830 00 90,casadejusticiaamalfi@casasdejusticia.gov.co,2020
2,ANTIOQUIA,ANORI,Anori,"Cra. 30 Nro. 30-42, parque principal.",(4) 8350473 - (4) 8350842 Ext. 116,casadejusticiaanori@casasdejusticia.gov.co,2020
3,ANTIOQUIA,APARTADO,Apartado,Calle 103 B No. 98 - 82 Barrio Chinita,No Registra,casadejusticiaapartado@casasdejusticia.gov.co,2020
4,ANTIOQUIA,BELLO,Bello,Diagonal 65 No. 42 A – 11 Barrio Niquía,6047944 ext. 1401,casadejusticiabello@casasdejusticia.gov.co,2020
...,...,...,...,...,...,...,...
155,BOGOTA,"BOGOTA, D.C.",Bogotá calle 45,Calle. 45 No. 13 – 16,(1) 3779595,casadejusticiacalle45@casasdejusticia.gov.co,2020
156,BOGOTA,"BOGOTA, D.C.",Bogotá Engativá,Transversal 113B No. 66 - 54,(1) 3779595,casadejusticiaengativa@casasdejusticia.gov.co,2020
157,BOGOTA,"BOGOTA, D.C.",Bogotá Bosa,"Carrera 81 D No. 59A-59 Sur, Barrio Argelia",(1) 3779595,casasdejusticiabogota@casasdejusticia.gov.co,2021
158,SAN ANDRES,SAN ANDRES,"CASA DE JUSTICIA SAN ANDRÉS, ISLA",Avenida las Américas Swand Ground,(8) 514-8227 ext 11-19 Fax:+ 5148699. 5130801 ...,casadejusticiasanandres@casasdejusticia.gov.co,2022


In [251]:
new_df["MUNICIPIO"].unique()

array(['LETICIA', 'AMALFI', 'ANORI', 'APARTADO', 'BELLO', 'CACERES',
       'CAUCASIA', 'CHIGORODO', 'EBEJICO', 'EL BAGRE',
       'EL CARMEN DE VIBORAL', 'ENVIGADO', 'FRONTINO', 'ITAGUI',
       'LA CEJA', 'MEDELLIN', 'NECHI', 'REMEDIOS', 'RIONEGRO',
       'SANTAFE DE ANTIOQUIA', 'SEGOVIA', 'SONSON', 'TARAZA', 'TURBO',
       'VEGACHI', 'YALI', 'YARUMAL', 'YONDO', 'ZARAGOZA', 'ARAUQUITA',
       'FORTUL', 'SARAVENA', 'BARANOA', 'BARRANQUILLA', 'GIRARDOT',
       'USIACURI', 'BOGOTA', 'CANTAGALLO', 'CARTAGENA', 'MAGANGUE',
       'SAN JACINTO', 'CHIQUINQUIRA', 'PAIPA', 'PUERTO BOYACA',
       'SOGAMOSO', 'TUNJA', 'VILLA DE LEYVA', 'LA DORADA', 'MANIZALES',
       'MARQUETALIA', 'RIOSUCIO', 'SUPIA', 'DONCELLO O EL DONCELLO',
       'SAN VICENTE DEL CAGUAN', 'FLORENCIA', 'LA SALINA', 'SACAMA',
       'NUNCHIA', 'PAZ DE ARIPORO', 'YOPAL', 'BUENOS AIRES', 'CALOTO',
       'CORINTO', 'GUACHENE', 'PIENDAMO', 'POPAYAN', 'PUERTO TEJADA',
       'SANTANDER DE QUILICHAO', 'TORIBIO', 'AGUACHICA'

## Fix residual errors of duplicated municipality names

There are duplicated names for San Andres, Tumaco, Bogota, el doncello and probably for cucuta and "CUCUTA  JUAN ATALAYA", but last one seems to be different.

In [252]:
# ISLA SAN ANDRES and SAN ANDRES 
condition = df_conditions(new_df, "MUNICIPIO", ["ISLA SAN ANDRES", "SAN ANDRES"])
new_df.loc[condition, "MUNICIPIO"] = "SAN ANDRES"

# TUMACO O SAN ANDRES DE TUMACO and TUMACO
condition = df_conditions(new_df, "MUNICIPIO", ["TUMACO", "TUMACO O SAN ANDRES DE TUMACO"])
new_df.loc[condition, "MUNICIPIO"] = "TUMACO"

# BOGOTA and BOGOTA, D.C.
condition = df_conditions(new_df, "MUNICIPIO", ["BOGOTA", "BOGOTA, D.C."])
new_df.loc[condition, "MUNICIPIO"] = "BOGOTA"

# DONCELLO, EL DONCELLO and DONCELLO O EL DONCELLO
condition = df_conditions(new_df, "MUNICIPIO", ["DONCELLO", "EL DONCELLO", "DONCELLO O EL DONCELLO"])
new_df.loc[condition, "MUNICIPIO"] = "EL DONCELLO"

In [253]:
tmp = new_df["MUNICIPIO"].unique()
tmp.sort()
tmp

array(['ABREGO', 'AGUACHICA', 'AMALFI', 'ANORI', 'APARTADO', 'ARAUQUITA',
       'ARMENIA', 'ATACO', 'BARANOA', 'BARRANCABERMEJA', 'BARRANCAS',
       'BARRANQUILLA', 'BELLO', 'BOGOTA', 'BUCARAMANGA', 'BUENAVENTURA',
       'BUENOS AIRES', 'CACERES', 'CAJICA', 'CALAMAR', 'CALI', 'CALOTO',
       'CANTAGALLO', 'CARMEN DEL DARIEN', 'CARTAGENA', 'CARTAGO',
       'CAUCASIA', 'CHAPARRAL', 'CHIA', 'CHIGORODO', 'CHIQUINQUIRA',
       'CIENAGA DE ORO', 'CONDOTO', 'CONVENCION', 'CORINTO', 'CUCUTA',
       'DOSQUEBRADAS', 'EBEJICO', 'EL BAGRE', 'EL CARMEN DE VIBORAL',
       'EL DONCELLO', 'ENVIGADO', 'ESPINAL', 'FLORENCIA', 'FLORIDA',
       'FLORIDABLANCA', 'FORTUL', 'FRONTINO', 'GIRARDOT', 'GUACHENE',
       'GUADALAJARA DE BUGA', 'IBAGUE', 'IPIALES', 'ISTMINA', 'ITAGUI',
       'LA CEJA', 'LA DORADA', 'LA PLATA', 'LA SALINA', 'LETICIA',
       'MAGANGUE', 'MANIZALES', 'MARQUETALIA', 'MEDELLIN', 'MOCOA',
       'MONTERIA', 'NECHI', 'NEIVA', 'NOVITA', 'NUNCHIA', 'OCAÑA',
       'ORTEGA', 'PAI

## Fix courthouses

In [254]:
new_df["CASA DE JUSTICIA"] = new_df["CASA DE JUSTICIA"].apply(remove_tildes)
new_df["CASA DE JUSTICIA"] = new_df["CASA DE JUSTICIA"].apply(str.upper)
new_df["CASA DE JUSTICIA"] = new_df["CASA DE JUSTICIA"].apply(lambda x: x.replace("-", ""))

new_df.drop_duplicates(
    ["CASA DE JUSTICIA", "DEPARTAMENTO", "MUNICIPIO"],
    inplace=True,
    ignore_index=True,
)


/tmp/ipykernel_39246/659205566.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df["CASA DE JUSTICIA"] = new_df["CASA DE JUSTICIA"].apply(remove_tildes)
/tmp/ipykernel_39246/659205566.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df["CASA DE JUSTICIA"] = new_df["CASA DE JUSTICIA"].apply(str.upper)
/tmp/ipykernel_39246/659205566.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the ca

In [255]:
from typing import Iterable
# There are somes courthouses that still that are duplicateds without "CASA DE JUSTICIA" at
# the start, so, if the courthouse the name of the courthouse isn't repeated, add to his name
# "CASA DE JUSTICIA" or "CENTRO DE CONVIVENCIA", but if it's found some duplicated,
# change name to the repeated one. Obviously, it's so hard to do this filter and innmediatly
# remove all duplicateds, but this it's just to facilitate the manual work
COURTHOUSE: str = "CASA DE JUSTICIA"
COEXISTENCECENTER: str = "CENTRO DE CONVIVENCIA" 


def change_courthouse_name(courthouse: str, courthouses: Iterable[str], email: str | None) -> str:
    def check(variation: str) -> str | None:
        new_name = f"{COEXISTENCECENTER} {variation}".strip()
        if new_name in courthouses:
            return new_name
        
        new_name = f"{COURTHOUSE} {variation}"
        if new_name in courthouses:
            return new_name
        
        return None


    if courthouse.startswith(COURTHOUSE) or \
        courthouse.startswith(COEXISTENCECENTER):
        return courthouse
    
    courthouse_list = courthouse.strip().split(" ")
    
    if (name := check(courthouse)):
        return name

    if (name := check(f'EL {courthouse}')):
        return name

    # If it's not in the dataset, then just add "CASA DE JUSTICIA"
    if len(courthouse_list) == 1:
        # For probability, it could be a courthouse
        if email is None:
            return f"{COURTHOUSE} {courthouse}"
        
        if email.startswith("centrodeconvivencia"):
            return f"{COEXISTENCECENTER} {courthouse}"
        
        return f"{COURTHOUSE} {courthouse}"

    variation = ''.join(courthouse_list[1:])
    if courthouse.startswith("EL") and (name := check(variation)):
        return name
    
    if email is None:
        return f"{COURTHOUSE} {courthouse}"
        
    if email.startswith("centrodeconvivencia"):
        return f"{COEXISTENCECENTER} {courthouse}"
    
    return f"{COURTHOUSE} {courthouse}"

In [256]:
courthouses = df["CASA DE JUSTICIA"].unique()
for courthouse in courthouses:
    email = (
        new_df.loc[df_conditions(new_df, "CASA DE JUSTICIA", [courthouse])]
        ["CORREOS INSTITUCIONALES"].to_numpy()
    )
    if courthouse == "CASA DE JUSTICIA TUMACO":
        email = email[1:]

    print(courthouse, email[0] if email else None)
    condition = df_conditions(new_df, "CASA DE JUSTICIA", [courthouse])
    new_df["CASA DE JUSTICIA"] = new_df["CASA DE JUSTICIA"].apply(
        lambda x: change_courthouse_name(x, courthouses, email[0] if email else None)
    )

/tmp/ipykernel_39246/1508196465.py:10: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  print(courthouse, email[0] if email else None)
/tmp/ipykernel_39246/1508196465.py:13: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  lambda x: change_courthouse_name(x, courthouses, email[0] if email else None)
/tmp/ipykernel_39246/1508196465.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df["CASA DE JUSTICIA"] = new_df["CASA DE JUSTICIA"].apply(


Leticia None
Amalfi None
Anori None
Apartado None
Bello None
Caceres None
Caucasia None
Chigorodo None
Ebejico None
El Bagre None
El Carmen de Viboral None
Envigado None
Frontino None
Itagui None
La Ceja None
Medellin Robledo None
Medellin Santo Domingo None
Medellin Villa del Socorro None
Medellin Veinte de Julio None
Medellin El Bosque None
Nechi None
Remedios None
Rionegro None
Santafe de Antioquia None
Segovia None
Sonsón None
Taraza None
Turbo None
Vegachi None
Yali None
Yarumal None
Yondo None
Zaragoza None
Arauquita None
Fortul None
Saravena None
Baranoa None
Barranquilla La Paz None
CASA DE JUSTICIA GIRARDOT casadejusticiagirardot@casasdejusticia.gov.co
Barranquilla Simón Bolívar None
Usiacurí None
BOGOTA Bosa None
BOGOTA Barrios Unidos None
BOGOTA Ciudad Bolívar None
BOGOTA Fontibón None
BOGOTA Kennedy None
BOGOTA Los Mártires None
BOGOTA San Cristóbal Sur None
BOGOTA Suba Pontevedra None
BOGOTA Suba la Campiña None
BOGOTA Usaquen None
BOGOTA calle 45 None
BOGOTA Engativá None

In [257]:
new_df.drop_duplicates(
    ["CASA DE JUSTICIA", "DEPARTAMENTO", "MUNICIPIO"],
    ignore_index=True,
    inplace=True
)

/tmp/ipykernel_39246/3724518716.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df.drop_duplicates(


In [258]:
new_df["CASA DE JUSTICIA"].unique()

array(['CASA DE JUSTICIA LETICIA', 'CASA DE JUSTICIA AMALFI',
       'CASA DE JUSTICIA ANORI', 'CASA DE JUSTICIA APARTADO',
       'CASA DE JUSTICIA BELLO', 'CASA DE JUSTICIA CACERES',
       'CASA DE JUSTICIA CAUCASIA', 'CASA DE JUSTICIA CHIGORODO',
       'CASA DE JUSTICIA EBEJICO', 'CASA DE JUSTICIA EL BAGRE',
       'CASA DE JUSTICIA EL CARMEN DE VIBORAL',
       'CASA DE JUSTICIA ENVIGADO', 'CASA DE JUSTICIA FRONTINO',
       'CASA DE JUSTICIA ITAGUI', 'CASA DE JUSTICIA LA CEJA',
       'CASA DE JUSTICIA MEDELLIN ROBLEDO',
       'CASA DE JUSTICIA MEDELLIN SANTO DOMINGO',
       'CASA DE JUSTICIA MEDELLIN VILLA DEL SOCORRO',
       'CASA DE JUSTICIA MEDELLIN VEINTE DE JULIO',
       'CASA DE JUSTICIA MEDELLIN EL BOSQUE', 'CASA DE JUSTICIA NECHI',
       'CASA DE JUSTICIA REMEDIOS', 'CASA DE JUSTICIA RIONEGRO',
       'CASA DE JUSTICIA SANTAFE DE ANTIOQUIA',
       'CASA DE JUSTICIA SEGOVIA', 'CASA DE JUSTICIA SONSON',
       'CASA DE JUSTICIA TARAZA', 'CASA DE JUSTICIA TURBO',
   

## Fix some errors manually

- CASA DE JUSTICIA EL CARMEN DE VIBORAL. Delete "EL"
- CASA DE JUSTICIA MEDELLIN 20 DE JULIO. Change "20" to "VEINTE"
- CASA DE JUSTICIA SANTAFE DE ANTIOQUIA. Change "SANTAFE" to "SANTA FE"
- CASA DE JUSTICIA SARAVENA. Change to "CENTRO DE CONVIVENCIA"
- CASA DE JUSTICIA BARANOA. Change to "CENTRO DE CONVIVENCIA"
- CASA DE JUSTICIA CANTAGALLO. Change to "CENTRO DE CONVIVENCIA"
- CASA DE JUSTICIA MAGANGUE. Change to "CENTRO DE CONVIVENCIA"
- CASA DE JUSTICIA SAN JACINTO. Change to "CENTRO DE CONVIVENCIA"
- CASA DE JUSTICIA PUERTO BOYACA. Change to "CENTRO DE CONVIVENCIA"
- CASA DE JUSTICIA LA DORADA. Change to "CENTRO DE CONVIVENCIA"
- CASA DE JUSTICIA MARQUETALIA. Change to "CENTRO DE CONVIVENCIA"
- CASA DE JUSTICIA LA SALINA. Change to "CENTRO DE CONVIVENCIA"
- CASA DE JUSTICIA NUNCHIA. Change to "CENTRO DE CONVIVENCIA"
- CASA DE JUSTICIA PIENDAMO. Change to "CENTRO DE CONVIVENCIA"
- CASA DE JUSTICIA AGUACHICA. Change to "CENTRO DE CONVIVENCIA"
- CASA DE JUSTICIA CALAMAR. Change to "CENTRO DE CONVIVENCIA"
- CASA DE JUSTICIA ABREGO. Change to "CENTRO DE CONVIVENCIA"
- CASA DE JUSTICIA OCAÑA. Change to "CENTRO DE CONVIVENCIA"
- CASA DE JUSTICIA TIBU. Change to "CENTRO DE CONVIVENCIA"
- CASA DE JUSTICIA VILLA DEL ROSARIO. Change to "CENTRO DE CONVIVENCIA"
- CASA DE JUSTICIA PUERTO CAICEDO. Change to "CENTRO DE CONVIVENCIA"
- CASA DE JUSTICIA BARRANCABERMEJA. Change to "CENTRO DE CONVIVENCIA"
- CASA DE JUSTICIA SAN GIL. Change to "CENTRO DE CONVIVENCIA"
- CASA DE JUSTICIA SINCE. Change to "CENTRO DE CONVIVENCIA"
- CASA DE JUSTICIA ESPINAL. Change to "CENTRO DE CONVIVENCIA"
- CASA DE JUSTICIA SAN ANDRES, ISLA. Delete ", ISLA"

After doing this, I realized that if I used better emails was a lot easier...

In [259]:
def change_to_coexistence_center(courthouse: str) -> str:
    return courthouse.replace(COURTHOUSE, COEXISTENCECENTER)

condition = df_conditions(
    new_df, "CASA DE JUSTICIA",
    ["CASA DE JUSTICIA SARAVENA", "CASA DE JUSTICIA BARANOA", "CASA DE JUSTICIA CANTAGALLO",
     "CASA DE JUSTICIA MAGANGUE", "CASA DE JUSTICIA SAN JACINTO", "CASA DE JUSTICIA PUERTO BOYACA",
     "CASA DE JUSTICIA LA DORADA", "CASA DE JUSTICIA MARQUETALIA", "CASA DE JUSTICIA LA SALINA",
     "CASA DE JUSTICIA NUNCHIA", "CASA DE JUSTICIA PIENDAMO", "CASA DE JUSTICIA AGUACHICA",
     "CASA DE JUSTICIA CALAMAR", "CASA DE JUSTICIA ABREGO", "CASA DE JUSTICIA OCAÑA",
     "CASA DE JUSTICIA TIBU", "CASA DE JUSTICIA VILLA DEL ROSARIO", "CASA DE JUSTICIA PUERTO CAICEDO",
     "CASA DE JUSTICIA BARRANCABERMEJA", "CASA DE JUSTICIA SAN GIL", "CASA DE JUSTICIA SINCE",
     "CASA DE JUSTICIA ESPINAL"]
)
new_df.loc[condition, "CASA DE JUSTICIA"] = new_df["CASA DE JUSTICIA"].apply(change_to_coexistence_center)

condition = df_conditions(new_df, "CASA DE JUSTICIA", ["CASA DE JUSTICIA EL CARMEN DE VIBORAL"])
new_df.loc[condition, "CASA DE JUSTICIA"] = new_df["CASA DE JUSTICIA"].apply(
    lambda x: x.replace("EL ", "")
)

condition = df_conditions(new_df, "CASA DE JUSTICIA", ["CASA DE JUSTICIA MEDELLIN 20 DE JULIO"])
new_df.loc[condition, "CASA DE JUSTICIA"] = new_df["CASA DE JUSTICIA"].apply(
    lambda x: x.replace("20", "VEINTE")
)

condition = df_conditions(new_df, "CASA DE JUSTICIA", ["CASA DE JUSTICIA SANTAFE DE ANTIOQUIA"])
new_df.loc[condition, "CASA DE JUSTICIA"] = new_df["CASA DE JUSTICIA"].apply(
    lambda x: x.replace("SANTAFE", "SANTA FE")
)

condition = df_conditions(new_df, "CASA DE JUSTICIA", ["CASA DE JUSTICIA SAN ANDRES, ISLA"])
new_df.loc[condition, "CASA DE JUSTICIA"] = new_df["CASA DE JUSTICIA"].apply(
    lambda x: x.replace(", ISLA", "")
)

## Add "CIUDADANA" to coexistence centers

In [260]:
# Add "CIUDADANA" to "CENTRO DE CONVIVENCIA"
def add_citizen(courthouse: str) -> str:
    citizen: str = "CIUDADANA"
    if courthouse.startswith(COURTHOUSE):
        return courthouse
    
    # There's only one coexistence center that ends with "CIUDADANA"
    # so add it nothing to it 
    if courthouse.endswith(citizen):
        return courthouse
    
    return courthouse.replace(COEXISTENCECENTER, f"{COEXISTENCECENTER} {citizen}")

In [261]:
new_df["CASA DE JUSTICIA"] = new_df["CASA DE JUSTICIA"].apply(add_citizen)

/tmp/ipykernel_39246/1841475340.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df["CASA DE JUSTICIA"] = new_df["CASA DE JUSTICIA"].apply(add_citizen)


In [263]:
new_df[["DEPARTAMENTO", "MUNICIPIO", "CASA DE JUSTICIA"]]. \
    to_csv("./processed/directorio_casas_de_justicia*.csv", index=False)